# Загрузка

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install natasha
!pip install navec
!pip install slovnet
!pip install ipymarkup

In [3]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
import nltk

In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('russian')
a = pd.read_csv('/content/drive/MyDrive/Russian_news_project/final_project/stop_words/стоп-слова.txt')
add_stop_words = [i for i in a['STOPWORDS_RUS'] if i not in sw]
sw = sw + add_stop_words
len(sw)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


649

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Russian_news_project/final_project/csv/final_df.csv', index_col=['Unnamed: 0'])

In [6]:
df.head()

,name,Date,text,krasivi_text,organizations,locations,persons,topic
0,Mash,2021-07-01,Человек своим телом закрыл старого друга семьи...,Человек своим телом закрыл старого друга семьи...,Ничего,Ничего,Ничего,3_ЧС
1,Mash,2021-07-01,Данные коронавирусу прошедшие новых случая За ...,Данные по коронавирусу за прошедшие сутки: 23 ...,Ничего,"Москва, Подмосковье",Ничего,5_Авиация и covid
2,Mash,2021-07-01,Яша артиллерист вчерашней прямой линии жаловал...,"""Яша-артиллерист"" со вчерашней прямой линии жа...","Яша-артиллерист, СУЭНКО","Тюменская область, Демьянка","Тарлан Тагиров, Яши-артиллериста, Тагиров",4_Россия и МИР
3,Mash,2021-07-01,В Москве врачи спасли летней женщине Два часа ...,В Москве врачи спасли жизнь 67-летней женщине....,Ничего,Москва,Ничего,3_ЧС
4,Mash,2021-07-01,Мы подумали В ресторане Якутске ребёнку налили...,"""Мы подумали, что это вода"" , 'В ресторане в Я...",Чочур Муран,Якутск,Ничего,1_Законопроекты


# Базовая предобработка текста

In [7]:
# Получить input для NER
text_NER = df['krasivi_text'].transform(lambda x: str(x))

In [8]:
text_NER

0         Человек своим телом закрыл старого друга семьи...
1         Данные по коронавирусу за прошедшие сутки: 23 ...
2         "Яша-артиллерист" со вчерашней прямой линии жа...
3         В Москве врачи спасли жизнь 67-летней женщине....
4         "Мы подумали, что это вода" , 'В ресторане в Я...
                                ...                        
102806    Галочка против фейков. «ВКонтакте» тестирует п...
102807    В Якутске трое мужчин попытались спасти ребенк...
102808    С московских причалов начали убирать зазывал. ...
102809    Девушка из Москвы перевела 30 миллионов рублей...
102810    «Сидим мы с братвами, б***ь, обсуждаем вас, ру...
Name: krasivi_text, Length: 102811, dtype: object

# **NER**

In [9]:
from navec import Navec
from slovnet import NER
from ipymarkup import show_span_ascii_markup as show_markup
from cmath import isnan

navec = Navec.load('/content/drive/MyDrive/Russian_news_project/final_project/navec_news_v1_1B_250K_300d_100q.tar')
ner = NER.load('/content/drive/MyDrive/Russian_news_project/final_project/slovnet_ner_news_v1.tar')
ner.navec(navec)

NER(
    infer=NERInfer(
        model=NER(
            emb=WordShapeEmbedding(
                word=NavecEmbedding(
                    id='news_v1_1B_250K_300d_100q',
                    indexes=Weight(
                        shape=[250002, 100],
                        dtype='uint8',
                        array=array([[176, 222, 248, ..., 244, 183, 191],
                               [215, 200, 168, ..., 120, 217,  21],
                               [ 83, 174,  54, ..., 106,  88, 251],
                               ...,
                               [133, 125, 123, ..., 124,  94,  24],
                               [183,  49, 180, ..., 151, 167,  68],
                               [255, 255, 255, ..., 255, 255, 255]], dtype=uint8)
                    ),
                    codes=Weight(
                        shape=[100, 256, 3],
                        dtype='float32',
                        array=array([[[ 0.01474531, -0.01112427,  0.35390645],
                         

In [10]:
#функция собирает организации (ORG), персоналии (PER) и локации (LOC)
def NER_mrkup(text):
    organizations = []
    
    persons = []
    
    locations = []

    markup = ner(text)
    
    spans = markup.spans
    
    for i in range(len(spans)):
    #собираем организации
        if spans[i].type == 'ORG':
            org_spans = spans[i]
            start, stop = org_spans.start, org_spans.stop
            organizations.append(str(text[start:stop]))
    #собираем персоналии
        if spans[i].type == 'PER':
            org_spans = spans[i]
            start, stop = org_spans.start, org_spans.stop
            persons.append(text[start:stop])
    #собираем локации
        if spans[i].type == 'LOC':
            org_spans = spans[i]
            start, stop = org_spans.start, org_spans.stop
            locations.append((text[start:stop]))    
    return organizations, persons, locations

In [11]:
result = text_NER.transform(lambda x: NER_mrkup(x))

In [12]:
result

0                                     ([], [], [Комсомола])
1                           ([], [], [Москве, Подмосковье])
2         ([Яша-артиллерист, СУЭНКО], [Тарлан Тагиров, Я...
3                                        ([], [], [Москве])
4                            ([Чочур Муран], [], [Якутске])
                                ...                        
102806                                ([ВКонтакте], [], [])
102807                                  ([], [], [Якутске])
102808    ([Центре организации дорожного движения], [], ...
102809    ([Ватсап], [Даниэлы, Даниэлы, Даниэла, Даниэло...
102810            ([], [Рая Мамедова, Раю, Раи], [Сургута])
Name: krasivi_text, Length: 102811, dtype: object

## Создаём таблицу с результатами NER

In [13]:
result_NER = result.to_frame().fillna(value = "nothing")

In [14]:
result_NER['organizations']=[i[0] for i in result_NER['krasivi_text']]
result_NER['persons']=[i[1] for i in result_NER['krasivi_text']]
result_NER['locations']=[i[2] for i in result_NER['krasivi_text']]
result_NER.drop(columns=['krasivi_text'],inplace=True)

In [15]:
result_NER

,organizations,persons,locations
0,[],[],[Комсомола]
1,[],[],"[Москве, Подмосковье]"
2,"[Яша-артиллерист, СУЭНКО]","[Тарлан Тагиров, Яши-артиллериста, Тагиров, Пу...","[Демьянка, Тюменская область]"
3,[],[],[Москве]
4,[Чочур Муран],[],[Якутске]
...,...,...,...
102806,[ВКонтакте],[],[]
102807,[],[],[Якутске]
102808,[Центре организации дорожного движения],[],[Москве-реке]
102809,[Ватсап],"[Даниэлы, Даниэлы, Даниэла, Даниэлой, Даниэлы,...","[Москвы, Одинцово]"


## Обрабатываем результаты NER

In [16]:
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger, NewsNERTagger, Doc
morph_vocab = MorphVocab()

In [17]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

In [18]:
def get_clean_data_NER(df_col): #подаём колонку из датафрейма с результатами NER (organization, persons, locations)
    q = []
    for a in df_col:
        doc = Doc(a)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        doc.tag_ner(ner_tagger)

        for span in doc.spans:
            span.normalize(morph_vocab)
            if [_.normal for _ in doc.spans] not in q:
                    q.append([_.normal for _ in doc.spans])
            else:
                continue
    return list(set([i[0] for i in q]))

### Обновляем таблицу NER

In [19]:
# создаем dataset NER

result_NER["locations"] = result_NER["locations"].map(lambda x: get_clean_data_NER(x))
result_NER["organizations"] = result_NER["organizations"].map(lambda x: get_clean_data_NER(x))
result_NER["persons"] = result_NER["persons"].map(lambda x: get_clean_data_NER(x))

result_NER["organizations_1"] = result_NER["organizations"].map(lambda x: " ".join(x).lower().split())
result_NER["locations_1"] = result_NER["locations"].map(lambda x: " ".join(x).lower().split())
result_NER["persons_1"] = result_NER["persons"].map(lambda x: " ".join(x).lower().split())

In [20]:
# result_NER.to_csv('/content/drive/MyDrive/Russian_news_project/final_project/NER/NER_2508.csv')

In [21]:
result_NER.head(2)

,organizations,persons,locations,organizations_1,locations_1,persons_1
0,[],[],[],[],[],[]
1,[],[],"[Подмосковье, Москва]",[],"[подмосковье, москва]",[]


In [22]:
# result_NER = pd.read_csv('/content/drive/MyDrive/Russian_news_project/final_project/NER/NER_2508.csv', index_col=['Unnamed: 0'])

### Создаём словарь NER + IDs

In [23]:
df["organizations_1"] = df["organizations"].map(lambda x: x.split(", "))

In [24]:
df["persons_1"] = df["persons"].map(lambda x: x.split(", "))

In [25]:
df["locations_1"] = df["locations"].map(lambda x: x.split(", "))

In [26]:
from collections import defaultdict
a= defaultdict(list)

def asd(row):
  global a
  for elem in row['persons_1']:
    a[elem].append(row.name)
  for elem in row['organizations_1']:
    a[elem].append(row.name)
  for elem in row['locations_1']:
    a[elem].append(row.name)

df[['persons_1', 'organizations_1', 'locations_1']].apply(asd, axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
102806    None
102807    None
102808    None
102809    None
102810    None
Length: 102811, dtype: object

In [27]:
dict_NER = {keys:values for keys, values in zip(a.keys(), a.values()) if len(values)>2}

In [28]:
# import joblib
# joblib.dump(dict_NER, '/content/drive/MyDrive/Russian_news_project/final_project/NER/dict_NER.pkl')

# Складываем оригинальный датасет + NER

In [ ]:
# df_new = pd.read_csv('/content/drive/MyDrive/Russian_news_project/final_project/csv/df_new_2508.csv',index_col=['Unnamed: 0'])

In [ ]:
# df_new.tail(25)

In [ ]:
# organizations = result_NER['organizations']
# locations = result_NER["locations"]
# persons = result_NER["persons"]
# df_new = pd.concat([df,organizations, locations, persons], axis = 1)

In [ ]:
# df_new.to_csv('/content/drive/MyDrive/Russian_news_project/final_project/csv/df_new_2508.csv')

# Сохраняем словари

In [ ]:
# import joblib

In [ ]:
# locations = joblib.load('/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/im/final_dict_loc.pkl')
# organizations = joblib.load('/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/im/final_dict_org.pkl')
# persons = joblib.load('/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/im/final_dict_per.pkl')

In [ ]:
# dict_NER = joblib.load('/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/dict_NER.pkl')

In [ ]:
# import joblib
# joblib.dump(dict_NER, '/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/dict_NER.pkl')

In [ ]:
# dict_NER_save = joblib.load("/content/drive/MyDrive/Russian_news_project/final_project/pyLDAvis_files/dict_NER.pkl")

In [ ]:
# df["text"][dict_NER["Евгений Ройзман"]]

14726    Для трехлетнего Марка Угрехелидзе СМА собрали ...
19122    Евгения Ройзмана вызвали полицию составления п...
19159    На бывшего мэра Екатеринбурга Евгения Ройзмана...
19205    Бывшего мэра Екатеринбурга Евгения Ройзмана ош...
21457    На Евгения Ройзмана составили протокол неуваже...
23391    Евгения Ройзмана оштрафовали рублей статье неу...
97144    На Евгения Ройзмана составили протокола статье...
Name: text, dtype: object

In [ ]:
# df["text"][dict_NER["Ройзман"]]

6563     Ты понятиям пытаешься предъявить Я губернаторо...
19159    На бывшего мэра Екатеринбурга Евгения Ройзмана...
23391    Евгения Ройзмана оштрафовали рублей статье неу...
85338    Почему независимой журналистики существует О л...
97833    Владимир Соловьев vs губернатор Свердловской о...
Name: text, dtype: object

In [ ]:
# len(df["text"].loc[dict_NER["путин"]].loc[(df['Date'] >= "2021-07-02") & (df['Date'] <= "2021-07-04")])

### Создаём списки слов из NER

In [ ]:
# sw_per = [x for list in result_NER["persons_1"].to_list() for x in list]
# sw_org = [x for list in result_NER["organizations_1"].to_list() for x in list]
# sw_loc = [x for list in result_NER["locations_1"].to_list() for x in list]

# sw_per2 = list(dict.fromkeys(sw_per))
# sw_org2 = list(dict.fromkeys(sw_org))
# sw_loc2 = list(dict.fromkeys(sw_loc))

# sw_NER = sw_org2 + sw_per2 + sw_loc2
# sw_NER = list(dict.fromkeys(sw_NER))

# import csv
# with open('//content/drive/MyDrive/Russian_news_project/final_project/stop_words/sw_NER.txt', 'w') as fp:
#   for i in sw_NER:
#     fp.write("%s\n" % i)